In [1]:
import progutil
from mnist_prediction_model import Auto_encoder, Prediction, Model
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_set = datasets.MNIST(root = "./data", train=True, download=True, transform=None)
test_set = datasets.MNIST(root = "./data", train=False, download = True, transform=None)

In [4]:
train_thresh, train_arrays, train_labels = progutil.pil_to_np(train_set)
test_thresh, test_arrays, test_labels = progutil.pil_to_np(test_set)

In [5]:
trObj = progutil.Data_Maps(train_arrays)
teObj = progutil.Data_Maps(test_arrays)

In [6]:
tr_features = {}
te_features = {}
for [i,j] in [[1,1], [1,2], [2,1], [2,2], [2,3], [3,2], [3,3]]:
    tr_features["maps{}_{}".format(i,j)] = trObj.get_maps(i,j)
    te_features["maps{}_{}".format(i,j)] = teObj.get_maps(i,j)

In [7]:
steps = [[1,1], [1,2], [2,1], [2,2], [2,3], [3,2], [3,3]]

def concat_features(features, steps):
    left = []
    right  = []
    top = []
    bottom = []
    
    for i,j in [[1,1], [1,2], [2,1], [2,2], [2,3], [3,2], [3,3]]:
        left.append(features["maps{}_{}".format(i,j)] [1])
        right.append(features["maps{}_{}".format(i,j)] [3])
        top.append(features["maps{}_{}".format(i,j)] [5] )
        bottom.append(features["maps{}_{}".format(i,j)] [7] )
    
    return torch.cat(left, dim = 1), torch.cat(right, dim = 1), torch.cat(top, dim = 1), torch.cat(bottom, dim = 1)


    

In [8]:
train_lefts, train_rights, train_tops, train_bottoms = concat_features(tr_features,steps)
test_lefts, test_rights, test_tops, test_bottoms = concat_features(te_features,steps)



In [9]:
batch_size = 400

trb_lefts, trb_rights, trb_tops, trb_bottoms = torch.split(train_lefts,batch_size), torch.split(train_rights,batch_size), torch.split(train_tops,batch_size), torch.split(train_bottoms,batch_size)
teb_lefts, teb_rights, teb_tops, teb_bottoms = torch.split(test_lefts,batch_size), torch.split(test_rights,batch_size), torch.split(test_tops,batch_size), torch.split(test_bottoms,batch_size)

train_labels = torch.FloatTensor(train_labels)
trb_labels = torch.split(train_labels, batch_size)

test_labels = torch.FloatTensor(test_labels)
teb_labels = torch.split(test_labels, batch_size)

In [10]:
train_embeddings = torch.split(torch.cat([train_lefts, train_rights, train_tops, train_bottoms ], dim = 1), batch_size)
test_embeddings = torch.split(torch.cat([test_lefts, test_rights, test_tops, test_bottoms], dim = 1), batch_size)

In [12]:
# input_size  = 76
# hidden_dims = [50, 7, 50]
output_size = 10
embedding_size = train_embeddings[0].shape[1]
emb_hidden_dims = [150, 30, 150]

Emb = Auto_encoder(embedding_size, emb_hidden_dims).to(device)
Emb_pred = Prediction(emb_hidden_dims[1], output_size).to(device)

linModel = Model(embedding_size, [128, 64], output_size ).to(device)

lr = 0.003
criterion = nn.MSELoss()
classerror = nn.CrossEntropyLoss()
# params = list(L.parameters()) + list(R.parameters()) + list(T.parameters()) + list(B.parameters()) + list(Emb.parameters()) + list(Emb_pred.parameters())
params = list(Emb_pred.parameters()) + list(Emb.parameters()) + list(linModel.parameters())
optimizer = torch.optim.Adam(params, lr)

print("The number of parameters in the Emb Auto Encoder -> {}".format(sum(p.numel() for p in Emb.parameters())))
print("The number of parameters in the Prediction layer -> {}".format(sum(p.numel() for p in Emb_pred.parameters())))
print("The number of parameters in the linmodel layer -> {}".format(sum(p.numel() for p in linModel.parameters())))

The number of parameters in the Emb Auto Encoder -> 151402
The number of parameters in the Prediction layer -> 310
The number of parameters in the linmodel layer -> 69450


In [13]:
Emb

Auto_encoder(
  (fclayer1): Linear(in_features=472, out_features=150, bias=True)
  (fclayer2): Linear(in_features=150, out_features=30, bias=True)
  (fclayer3): Linear(in_features=30, out_features=150, bias=True)
  (fclayer4): Linear(in_features=150, out_features=472, bias=True)
  (lerelu): LeakyReLU(negative_slope=0.2)
)

In [14]:
EPOCHS = 100

for epoch in range(EPOCHS):
    print("Epoch-{}/{}---------->".format(epoch+1, EPOCHS))
    correct = 0
    catloss = 0
    for i in range(len(trb_labels)):
        optimizer.zero_grad()
        output = linModel(train_embeddings[i].cuda())
        labs = trb_labels[i].type(torch.LongTensor).cuda()
        closs = classerror(output, labs )
        closs.backward()
        optimizer.step()
        catloss += closs.item()
        correct += (torch.argmax(output, dim = 1) == labs).float().sum()
    print("Epoch loss = {} -------- Accuracy = {}".format(catloss, correct/(batch_size * len(train_embeddings))))

Epoch-1/100---------->
Epoch loss = 270.35851633548737 -------- Accuracy = 0.6829833388328552
Epoch-2/100---------->
Epoch loss = 250.01448106765747 -------- Accuracy = 0.7973833680152893
Epoch-3/100---------->
Epoch loss = 245.0395554304123 -------- Accuracy = 0.8306500315666199
Epoch-4/100---------->
Epoch loss = 236.3610544204712 -------- Accuracy = 0.8894500136375427
Epoch-5/100---------->
Epoch loss = 233.96314907073975 -------- Accuracy = 0.9038333296775818
Epoch-6/100---------->
Epoch loss = 232.69462394714355 -------- Accuracy = 0.9125166535377502
Epoch-7/100---------->
Epoch loss = 231.66963016986847 -------- Accuracy = 0.9188500046730042
Epoch-8/100---------->
Epoch loss = 230.6615490913391 -------- Accuracy = 0.9255000352859497
Epoch-9/100---------->
Epoch loss = 230.1511708498001 -------- Accuracy = 0.9284499883651733
Epoch-10/100---------->
Epoch loss = 229.5373581647873 -------- Accuracy = 0.9329000115394592
Epoch-11/100---------->
Epoch loss = 228.98301100730896 --------

In [15]:
with torch.no_grad():
    test_corrects = 0
    test_catloss = 0
    for i in range(len(teb_labels)):
        test_output = linModel(test_embeddings[i].cuda())
        test_labs = teb_labels[i].type(torch.LongTensor).cuda()
        test_closs = classerror(test_output, test_labs )
        test_catloss += test_closs.item()
        test_corrects += (torch.argmax(test_output, dim = 1) == test_labs).float().sum()
    print("Test Set -------- Accuracy = {}".format(test_catloss, test_corrects/(batch_size * len(test_embeddings)))) 

Test Set -------- Accuracy = 38.18976020812988


In [17]:
test_output.shape

torch.Size([400, 10])

In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
    print("Epoch-{}/{}---------->".format(epoch+1, EPOCHS))
    elloss = 0
    erloss = 0
    etloss = 0
    ebloss = 0
    EmbLoss = 0
    
    for ebatch in train_embeddings:
        optimizer.zero_grad()
        _,eout = Emb(ebatch.cuda())
        eloss = criterion(ebatch.cuda(), eout)
        eloss.backward()
        optimizer.step()
        EmbLoss += eloss.item()
    print("Emb loss = {}".format(EmbLoss))
        
    
    

In [ ]:
EPOCHS = 200

for epoch in range(EPOCHS):
    print("Epoch-{}/{}---------->".format(epoch+1, EPOCHS))
    correct = 0
    catloss = 0
    for i in range(len(trb_labels)):
        optimizer.zero_grad()
        enc,_ = Emb(train_embeddings[i].cuda())
        output = Emb_pred(enc)
        labs = trb_labels[i].type(torch.LongTensor).cuda()
        closs = classerror(output, labs )
        closs.backward()
        optimizer.step()
        catloss += closs.item()
        correct += (torch.argmax(output, dim = 1) == labs).float().sum()
    print("Epoch loss = {} -------- Accuracy = {}".format(catloss, correct/(batch_size * len(train_embeddings))))
        
    

In [21]:
with torch.no_grad():
    test_corrects = 0
    test_catloss = 0
    for i in range(len(teb_labels)):
        test_enc,_ = Emb(test_embeddings[i].cuda())
        test_output = Emb_pred(test_enc)
        test_labs = teb_labels[i].type(torch.LongTensor).cuda()
        test_closs = classerror(test_output, test_labs )
        test_catloss += test_closs.item()
        test_corrects += (torch.argmax(test_output, dim = 1) == test_labs).float().sum()
    print("Test Set -------- Accuracy = {}".format(test_catloss, test_corrects/(batch_size * len(test_embeddings)))) 

Test Set -------- Accuracy = 38.91535210609436


torch.Size([400, 472])